# arXiv additional analysis

This notebook contains some additional, final analysis for our arXiv mapping project. We are integrating them here because the main analytical notebook has become too big and complex (we should disentangle it too)

## Preamble

In [ ]:
%run notebook_preamble.ipy

import geopandas as gp

## Action 1: arXiv files in Import AI

How many links to arXiv papers can we find in import AI?


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
#Download the first page from import AI
iai = requests.get('https://jack-clark.net/')

In [ ]:
#Parse
soup = BeautifulSoup(iai.content)

In [ ]:
#Get links and remove non documents
links = [l.get('href') for l in soup.find_all('a')]

links_clean = [x for x in links if not any(nope in x for nope in ['jack-clark','twitter','wordpress','github'])]

#How many are to arXiv?
arx = np.mean(['arxiv' in x for x in links_clean])

arx

## Action 2: Visualise AI geographies

In [ ]:
countries = gp.read_file('../data/external/countries/Countries_WGS84.shp')

In [ ]:
grid_geo = pd.read_csv('../data/external/17_8_2019_papers_institution_ucl_cleaned.csv',compression='zip',dtype={'article_id':str})

In [ ]:
analysis_df = pd.read_csv('../data/processed/18_8_2019_analysis_fin.csv',compression='zip',dtype={'paper_id':str})

In [ ]:
arx_all = pd.read_csv('../data/processed/1_8_2019_arxiv_enriched.csv',compression='zip',dtype={'id':str})

### Label Grid GEO 

In [ ]:
is_ai = set(list(arx_all.loc[arx_all['is_ai']==True]['id']))

In [ ]:
notable_topics = [
    #'face-faces-identity-face_recognition-facial','person-surveillance-persons-pedestrian-pedestrians',
    #'attacks-attack-adversary-vulnerable-threat',
    #'emotions-emotion-neutral-emotional-spontaneous',
    'reinforcement_learning-policy-policies-reward-deep_reinforcement_learning',
    'cnn-convolutional_neural_networks-cnns-convolutional_neural_network-convolutional_neural_network_cnn',
    'training-trained-deep_learning-deep-train',
    'generator-gan-discriminator-generative_adversarial_networks_gans-gans',
    'translation-neural_machine_translation-machine_translation-translate-translations',
    'recurrent-lstm-rnn-recurrent_neural_network-recurrent_neural_networks']

In [ ]:
is_notable = set(list([row['paper_id'] for pid, row in analysis_df.iterrows() if any(x>0.05 for x in row[notable_topics])]))

In [ ]:
grid_geo['is_ai'],grid_geo['is_notable_ai'] = [[x in this_set for x in grid_geo['article_id']] for this_set in [is_ai,is_notable]]

In [ ]:
fig,ax = plt.subplots(figsize=(12,7))

my_grid = grid_geo.loc[(grid_geo['is_ai']==True)]

#my_grid = grid_geo

ax.hexbin(my_grid['institute_lon'],my_grid['institute_lat'],cmap='Reds',bins=10,gridsize=40,edgecolor='lightgrey',linewidth=0.1)

In [ ]:
# fig,ax = plt.subplots(figsize=(18,12))

# my_grid = grid_geo.loc[(grid_geo['is_ai']==True)]

# ax.hexbin(my_grid['institute_lon'],my_grid['institute_lat'],cmap='Reds',bins=10,gridsize=40,edgecolor='lightgrey',linewidth=0.07)

# #countries.plot(ax=ax,color=(0,0,0,0),edgecolor='lightgrey',linewidth=0.5)
